# Fuzzy cognitive diagnosis framework (FuzzyCDF)

This notebook will show you how to train and use the FuzzyCDF.
First, we will show how to get the data (here we use Math1 from math2015 as the dataset).
Then we will show how to train a FuzzyCDF and perform the parameters persistence.
At last, we will show how to load the parameters from the file and evaluate on the test dataset.

The script version could be found in [FuzzyCDF.py](FuzzyCDF.ipynb)

## Data Preparation

Before we process the data, we need to first acquire the dataset which is shown in [prepare_dataset.ipynb](prepare_dataset.ipynb)

In [8]:
# Load the data from files
import numpy as np
import json

data_set_list=('FrcSub','Math1', 'Math2')
data_set_name=data_set_list[2]
if data_set_name == 'FrcSub':
    read_dir='../data/frcSub/'
elif data_set_name == 'Math1':
    read_dir='../data/math1/'
elif data_set_name == 'Math2':
    read_dir='../data/math2/'

# type of problems
obj_prob_index = np.loadtxt(read_dir+"obj_prob_index.csv", delimiter=',', dtype=int)
sub_prob_index = np.loadtxt(read_dir+"sub_prob_index.csv", delimiter=',', dtype=int)
# Q matrix
q_m = np.loadtxt(read_dir+'q_m.csv', dtype=int, delimiter=',')
prob_num, know_num = q_m.shape[0], q_m.shape[1]

# training data
with open(read_dir+'train.json', encoding='utf-8') as file:
    train_set = json.load(file)
stu_num = max([x['user_id'] for x in train_set]) + 1
R = -1 * np.ones(shape=(stu_num, prob_num))
for log in train_set:
    R[log['user_id'], log['item_id']] = log['score']

# testing data
with open(read_dir+'test.json', encoding='utf-8') as file:
    test_set = json.load(file)

In [9]:
print(train_set[0], test_set[0])

{'user_id': 0, 'item_id': 0, 'score': 1.0} {'user_id': 0, 'item_id': 15, 'score': 0.0}


In [10]:
len(train_set), len(test_set)

(62576, 15644)

## Training and Persistence

In [11]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [12]:
from FuzzyCDF import FuzzyCDF

cdm = FuzzyCDF(R, q_m, stu_num, prob_num, know_num, obj_prob_index, sub_prob_index, skip_value=-1)

cdm.train(epoch=10, burnin=5)
cdm.save("fuzzycdf.params")

INFO:root:save parameters to fuzzycdf.params


## Loading and Testing

In [13]:
cdm.load("fuzzycdf.params")
print('data_set_name:',data_set_name)
if len(sub_prob_index)>0:
    (obj_acc,obj_auc,obj_rmse,obj_mae),(sub_rmse,sub_mae)= cdm.eval(test_set)
    print("obj_acc: %.6f,obj_auc: %.6f,obj_rmse: %.6f, obj_mae: %.6f,\nsub_rmse: %.6f, sub_mae: %.6f"% (
        obj_acc,obj_auc,obj_rmse,obj_mae,sub_rmse,sub_mae))
else:
    obj_acc,obj_auc,obj_rmse,obj_mae=cdm.eval(test_set)
    print("obj_acc: %.6f,obj_auc: %.6f,obj_rmse: %.6f, obj_mae: %.6f" % (
        obj_acc,obj_auc,obj_rmse,obj_mae))

INFO:root:load parameters from fuzzycdf.params


data_set_name: Math2


evaluating: 100%|██████████| 15644/15644 [00:00<00:00, 500676.76it/s]

obj_acc: 0.621680,obj_auc: 0.671631,obj_rmse: 0.483727, obj_mae: 0.445671,
sub_rmse: 0.365104, sub_mae: 0.312644


## Incremental Training

In [14]:
new_data = [{'user_id': 0, 'item_id': 2, 'score': 0.0}, {'user_id': 1, 'item_id': 1, 'score': 1.0}]
cdm.inc_train(new_data, epoch=10, burnin=5)